In [1]:
# sequence_representation.py
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from collections import defaultdict
import json
import pickle
from tqdm import tqdm

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Load and examine the data
df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])

print("\n" + "="*80)
print("DATA EXPLORATION")
print("="*80)
print(f"Total quotes: {len(df_quotes):,}")
print(f"Unique customers: {df_quotes['numero_compte'].nunique():,}")
print(f"Date range: {df_quotes['dt_creation_devis'].min()} to {df_quotes['dt_creation_devis'].max()}")

# Show columns
print("\n📋 Available columns:")
print(df_quotes.columns.tolist())

# Sample of data
print("\n📊 Sample data (first 3 quotes):")
print(df_quotes.head(3).to_string())

PyTorch version: 2.9.1+cpu
CUDA available: False

DATA EXPLORATION
Total quotes: 34,014
Unique customers: 23,888
Date range: 2023-01-02 00:00:00 to 2026-01-22 00:00:00

📋 Available columns:
['id_devis', 'num_devis', 'nom_devis', 'nom_agence', 'nom_filiale_zone', 'nom_region', 'statut_devis', 'fg_devis_emis', 'fg_devis_refuse', 'fg_devis_accepte', 'dt_creation_devis', 'dt_signature_devis', 'fg_3_mois_mature', 'type_devis', 'mt_apres_remise_ht_devis', 'mt_marge', 'nb_devis_emis', 'mt_apres_remise_ht_emis_devis', 'mt_marge_emis_devis', 'mt_remise_exceptionnelle_ht', 'mt_ttc_apres_aide_devis', 'mt_ttc_avant_aide_devis', 'mt_prime_cee', 'mt_prime_maprimerenov', 'fg_activite_commerciale', 'prenom_nom_createur', 'prenom_nom_commercial', 'nom_campagne', 'famille_equipement_produit', 'type_equipement_produit', 'dth_emission_devis', 'dt_emission_calcule_devis', 'id_opportunite', 'fg_devis_principal', 'lb_statut_preparation_chantier', 'numero_compte', 'dt_visite_commerciale', 'fonction_commercial

In [2]:
def test_conversion_sequence(df):
    """
    Test if customers continue getting quotes after conversion
    """
    print("=" * 80)
    print("VALIDATING: Do customers get more quotes after conversion?")
    print("=" * 80)
    
    # Sort by customer and date
    df = df.sort_values(['numero_compte', 'dt_creation_devis']).copy()
    
    results = {
        'total_customers': df['numero_compte'].nunique(),
        'customers_with_conversion': 0,
        'customers_with_post_conversion_quotes': 0,
        'total_conversions': 0,
        'post_conversion_quotes': 0
    }
    
    for customer_id, customer_data in df.groupby('numero_compte'):
        # Sort chronologically
        customer_data = customer_data.sort_values('dt_creation_devis')
        conversions = customer_data['fg_devis_accepte'].values
        
        # Find first conversion
        conversion_indices = np.where(conversions == 1)[0]
        
        if len(conversion_indices) > 0:
            results['customers_with_conversion'] += 1
            results['total_conversions'] += len(conversion_indices)
            
            first_conversion_idx = conversion_indices[0]
            
            # Check if there are quotes AFTER first conversion
            if first_conversion_idx < len(customer_data) - 1:
                results['customers_with_post_conversion_quotes'] += 1
                post_count = len(customer_data) - first_conversion_idx - 1
                results['post_conversion_quotes'] += post_count
    
    return results

test_conversion_sequence(df_quotes)

VALIDATING: Do customers get more quotes after conversion?


{'total_customers': 23888,
 'customers_with_conversion': 9458,
 'customers_with_post_conversion_quotes': 1177,
 'total_conversions': 10204,
 'post_conversion_quotes': np.int64(1701)}

In [3]:
def analyze_conversion_behavior(df):
    """
    Analyze customer behavior before/after conversion
    """
    print("\n📊 CONVERSION BEHAVIOR ANALYSIS")
    print("-" * 60)
    
    df = df.sort_values(['numero_compte', 'dt_creation_devis']).copy()
    
    pre_conversion_stats = []
    post_conversion_stats = []
    
    for customer_id, customer_data in df.groupby('numero_compte'):
        customer_data = customer_data.sort_values('dt_creation_devis')
        dates = customer_data['dt_creation_devis'].values
        conversions = customer_data['fg_devis_accepte'].values
        
        # Find first conversion
        conversion_indices = np.where(conversions == 1)[0]
        
        if len(conversion_indices) > 0:
            first_conv_idx = conversion_indices[0]
            
            # Quotes BEFORE conversion
            if first_conv_idx > 0:
                pre_dates = dates[:first_conv_idx]
                if len(pre_dates) > 1:
                    pre_gaps = np.diff(pre_dates).astype('timedelta64[D]').astype(int)
                    pre_conversion_stats.extend(pre_gaps)
            
            # Quotes AFTER conversion
            if first_conv_idx < len(dates) - 1:
                post_dates = dates[first_conv_idx + 1:]
                if len(post_dates) > 0:
                    # FIX: Convert to days properly
                    conv_to_next = (post_dates[0] - dates[first_conv_idx]).astype('timedelta64[D]').astype(int)
                    post_conversion_stats.append(conv_to_next)
                    
                    if len(post_dates) > 1:
                        post_gaps = np.diff(post_dates).astype('timedelta64[D]').astype(int)
                        post_conversion_stats.extend(post_gaps)
    
    # Print results
    if pre_conversion_stats:
        print(f"Pre-conversion time gaps:")
        print(f"  Count: {len(pre_conversion_stats):,}")
        print(f"  Mean: {np.mean(pre_conversion_stats):.1f} days")
        print(f"  Median: {np.median(pre_conversion_stats):.1f} days")
        print(f"  Std: {np.std(pre_conversion_stats):.1f} days")
    
    if post_conversion_stats:
        print(f"\nPost-conversion time gaps:")
        print(f"  Count: {len(post_conversion_stats):,}")
        print(f"  Mean: {np.mean(post_conversion_stats):.1f} days")
        print(f"  Median: {np.median(post_conversion_stats):.1f} days")
        print(f"  Std: {np.std(post_conversion_stats):.1f} days")

analyze_conversion_behavior(df_quotes)


📊 CONVERSION BEHAVIOR ANALYSIS
------------------------------------------------------------
Pre-conversion time gaps:
  Count: 831
  Mean: 48.1 days
  Median: 0.0 days
  Std: 127.7 days

Post-conversion time gaps:
  Count: 1,701
  Mean: 86.0 days
  Median: 13.0 days
  Std: 154.8 days


In [4]:
def analyze_conversion_by_position(df):
    """
    Analyze which quote in sequence gets converted
    """
    print("\n🎯 CONVERSION BY QUOTE POSITION")
    print("-" * 60)
    
    conversion_by_position = {}
    total_customers = df['numero_compte'].nunique()
    
    for customer_id, customer_data in df.groupby('numero_compte'):
        customer_data = customer_data.sort_values('dt_creation_devis')
        conversions = customer_data['fg_devis_accepte'].values
        
        # Find which quote(s) converted
        conv_positions = np.where(conversions == 1)[0] + 1  # 1-indexed
        
        for pos in conv_positions:
            conversion_by_position[pos] = conversion_by_position.get(pos, 0) + 1
    
    # Sort by position
    positions = sorted(conversion_by_position.keys())
    
    print(f"Total customers: {total_customers:,}")
    print(f"\nFirst conversion occurs at quote #:")
    
    for pos in positions[:10]:  # Show first 10 positions
        count = conversion_by_position[pos]
        percent = (count / total_customers) * 100
        print(f"  {pos}: {count:,} customers ({percent:.1f}%)")
    
    if len(positions) > 10:
        print(f"  ... and {len(positions) - 10} more positions")
    
    # Calculate cumulative percentage
    print(f"\n📊 CUMULATIVE DISTRIBUTION:")
    cumulative = 0
    for pos in sorted(conversion_by_position.keys()):
        cumulative += conversion_by_position[pos]
        percent = (cumulative / total_customers) * 100
        print(f"  By quote #{pos}: {cumulative:,} customers ({percent:.1f}%)")
        if percent >= 95:
            print(f"  → 95% of conversions happen by quote #{pos}")
            break

analyze_conversion_by_position(df_quotes)


🎯 CONVERSION BY QUOTE POSITION
------------------------------------------------------------
Total customers: 23,888

First conversion occurs at quote #:
  1: 7,105 customers (29.7%)
  2: 2,194 customers (9.2%)
  3: 555 customers (2.3%)
  4: 201 customers (0.8%)
  5: 83 customers (0.3%)
  6: 43 customers (0.2%)
  7: 12 customers (0.1%)
  8: 4 customers (0.0%)
  9: 2 customers (0.0%)
  10: 1 customers (0.0%)
  ... and 4 more positions

📊 CUMULATIVE DISTRIBUTION:
  By quote #1: 7,105 customers (29.7%)
  By quote #2: 9,299 customers (38.9%)
  By quote #3: 9,854 customers (41.3%)
  By quote #4: 10,055 customers (42.1%)
  By quote #5: 10,138 customers (42.4%)
  By quote #6: 10,181 customers (42.6%)
  By quote #7: 10,193 customers (42.7%)
  By quote #8: 10,197 customers (42.7%)
  By quote #9: 10,199 customers (42.7%)
  By quote #10: 10,200 customers (42.7%)
  By quote #12: 10,201 customers (42.7%)
  By quote #15: 10,202 customers (42.7%)
  By quote #17: 10,203 customers (42.7%)
  By quote #1

In [5]:
def analyze_multiple_conversions(df):
    """
    Check if customers have multiple converted quotes
    """
    print("\n🔍 MULTIPLE CONVERSIONS ANALYSIS")
    print("-" * 60)
    
    multiple_conv_customers = 0
    conversion_counts = {}
    
    for customer_id, customer_data in df.groupby('numero_compte'):
        conversion_count = customer_data['fg_devis_accepte'].sum()
        
        if conversion_count > 1:
            multiple_conv_customers += 1
            conversion_counts[conversion_count] = conversion_counts.get(conversion_count, 0) + 1
    
    total_customers = df['numero_compte'].nunique()
    
    print(f"Customers with >1 conversion: {multiple_conv_customers:,}")
    print(f"Percentage: {(multiple_conv_customers/total_customers*100):.2f}%")
    
    if conversion_counts:
        print("\nBreakdown of multiple conversions:")
        for count, customers in sorted(conversion_counts.items()):
            print(f"  {count} conversions: {customers:,} customers")

analyze_multiple_conversions(df_quotes)


🔍 MULTIPLE CONVERSIONS ANALYSIS
------------------------------------------------------------
Customers with >1 conversion: 626
Percentage: 2.62%

Breakdown of multiple conversions:
  2.0 conversions: 537 customers
  3.0 conversions: 67 customers
  4.0 conversions: 16 customers
  5.0 conversions: 4 customers
  6.0 conversions: 1 customers
  7.0 conversions: 1 customers


In [6]:
# Run all validation tests
print("🚀 RUNNING DATA VALIDATION TESTS")
print("=" * 80)

# Test 1
results = test_conversion_sequence(df_quotes)
print(f"\n📋 TEST 1: Sequential Conversion Pattern")
print(f"  Total customers: {results['total_customers']:,}")
print(f"  Customers with at least 1 conversion: {results['customers_with_conversion']:,}")
print(f"  Customers with quotes AFTER conversion: {results['customers_with_post_conversion_quotes']:,}")
print(f"  Percentage: {(results['customers_with_post_conversion_quotes']/max(1, results['customers_with_conversion'])*100):.1f}%")
print(f"  Total post-conversion quotes: {results['post_conversion_quotes']:,}")

# Test 2
analyze_conversion_behavior(df_quotes)

# Test 3
analyze_conversion_by_position(df_quotes)

# Test 4
analyze_multiple_conversions(df_quotes)

🚀 RUNNING DATA VALIDATION TESTS
VALIDATING: Do customers get more quotes after conversion?

📋 TEST 1: Sequential Conversion Pattern
  Total customers: 23,888
  Customers with at least 1 conversion: 9,458
  Customers with quotes AFTER conversion: 1,177
  Percentage: 12.4%
  Total post-conversion quotes: 1,701

📊 CONVERSION BEHAVIOR ANALYSIS
------------------------------------------------------------
Pre-conversion time gaps:
  Count: 831
  Mean: 48.1 days
  Median: 0.0 days
  Std: 127.7 days

Post-conversion time gaps:
  Count: 1,701
  Mean: 86.0 days
  Median: 13.0 days
  Std: 154.8 days

🎯 CONVERSION BY QUOTE POSITION
------------------------------------------------------------
Total customers: 23,888

First conversion occurs at quote #:
  1: 7,105 customers (29.7%)
  2: 2,194 customers (9.2%)
  3: 555 customers (2.3%)
  4: 201 customers (0.8%)
  5: 83 customers (0.3%)
  6: 43 customers (0.2%)
  7: 12 customers (0.1%)
  8: 4 customers (0.0%)
  9: 2 customers (0.0%)
  10: 1 customers 